In [1]:
# go on the specific directory picture by pcture and label pictures

In [2]:
import os
import shutil
import json
import cv2
import pyheif
from PIL import Image

In [3]:
def load_labels(label_file):
    """Load existing labels from a JSON file."""
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            return json.load(f)
    return {}

In [4]:
def convert_heic_to_jpg(heic_path):
    """Convert HEIC image to JPG format for processing."""
    heif_file = pyheif.read(heic_path)
    image = Image.frombytes(
        heif_file.mode, heif_file.size, heif_file.data,
        "raw", heif_file.mode, heif_file.stride
    )
    jpg_path = heic_path.rsplit(".", 1)[0] + ".jpg"
    image.save(jpg_path, "JPEG")
    return jpg_path

In [5]:
def save_labels(label_file, labels):
    """Save labels to a JSON file."""
    with open(label_file, 'w') as f:
        json.dump(labels, f, indent=4)

In [6]:
def label_images(source_dir, output_dir, label_file="labels.json"):
    """Go through images, ask for multiple labels, and store them in labeled directories."""
    labels = load_labels(label_file)
    os.makedirs(output_dir, exist_ok=True)
    
    for root, _, files in os.walk(source_dir):
        for file in files:
            file_path = os.path.join(root, file)
            
            if file.lower().endswith(".heic"):
                print(f"Converting HEIC image: {file}")
                file_path = convert_heic_to_jpg(file_path)
            
            if file_path.lower().endswith((".jpg", ".jpeg", ".png", ".gif", ".tiff")):
                if file_path in labels:
                    print(f"Skipping {file} (already labeled as {labels[file_path]})")
                    continue  # Skip to the next file
                
                if not os.path.exists(file_path):
                    print(f"Error: File not found - {file_path}")
                    continue  # Skip to the next file
                
                # Show the image
                image = cv2.imread(file_path)
                
                if image is None:
                    print(f"⚠️ Warning: OpenCV could not read {file_path}. Skipping.")
                    continue  # Skip to the next file
                
                cv2.imshow("Labeling", image)
                cv2.waitKey(1)  # Small delay to refresh window
                
                label_input = input(f"Enter labels for {file} (comma-separated): ")
                labels_list = [label.strip() for label in label_input.split(",") if label.strip()]
                
                if not labels_list:
                    print(f"Skipping {file}")
                    cv2.destroyAllWindows()
                    continue
                
                labels[file_path] = labels_list
                
                for label in labels_list:
                    # Create directory for each label
                    label_folder = os.path.join(output_dir, label)
                    os.makedirs(label_folder, exist_ok=True)
                    
                    # Copy the image to the labeled directories
                    shutil.copy(file_path, os.path.join(label_folder, os.path.basename(file_path)))
                    print(f"Saved {file} to {label_folder}")
                
                # Save labels periodically
                save_labels(label_file, labels)
                
                cv2.destroyAllWindows()
    
    print("Labeling complete!")
    save_labels(label_file, labels)


In [ ]:
#if __name__ == "__main__":
SOURCE_DIR = "/Volumes/NFP4TBSSD/PHOTOS_ORGANIZE/CameraUploads"
OUTPUT_DIR = "/Volumes/NFP4TBSSD/organized_photos/organized_photos_labeled/_val"
LABEL_FILE = "../data/labels.json"
    
label_images(SOURCE_DIR, OUTPUT_DIR, LABEL_FILE)

⚠️ Warning: OpenCV could not read /Volumes/NFP4TBSSD/PHOTOS_ORGANIZE/CameraUploads/2021-05-19 18.30.49.jpg. Skipping.
Skipping 2017-09-14 08.47.14.jpg
Skipping 2017-08-12 16.35.40.jpg
Skipping 2018-11-01 13.20.35-1.jpg
Skipping 2019-11-23 08.37.31.jpg
Skipping 2017-08-09 20.05.53.jpg
Skipping 2022-04-07 13.11.40.jpg
Skipping 2018-11-11 17.16.00.jpg
Skipping 2019-10-12 19.08.40.jpg
Skipping 2017-08-09 20.11.26.jpg (already labeled as ['Amelia', 'Nuno'])
Skipping 2021-09-22 18.32.52.png
Skipping 2017-10-28 18.00.09.jpg (already labeled as ['Gato'])
Skipping 2019-11-04 08.44.52-3.jpg
Skipping 2024-08-09 10.51.32.jpg (already labeled as ['Amelia'])
Converting HEIC image: 2020-09-06 19.15.32.heic


error: struct heif_decoding_options: wrong total size (cdef says 72, but C compiler says 80). fix it or use "...;" as the last field in the cdef for struct heif_decoding_options to make it flexible

: 